In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import numpy as np
import pandas as pd
import exrex
import random
from sklearn.model_selection import KFold

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def generate_matching_strings(regex, num_samples, seq_length):
    # Generate strings that match the regular expression
    generated_strings = [exrex.getone(regex) for _ in range(num_samples)]
    generated_strings = [s[:seq_length] for s in generated_strings]
    return generated_strings

def generate_random_strings(alphabet, num_samples, seq_length):
    # Generate strings of specified length over the specified alphabet
    alphabet = ''.join(alphabet)
    res = [''.join(random.choices(alphabet, k=random.randint(0, seq_length))) for _ in range(num_samples)]
    return res

def generate_data(regex, alphabet, num_samples, seq_length, correct_proportion):
    num_correct = int(num_samples * correct_proportion)
    num_incorrect = num_samples - num_correct

    # Generate the correct and incorrect strings
    correct_data = generate_matching_strings(regex, num_correct, seq_length)
    incorrect_data = generate_random_strings(alphabet, num_incorrect, seq_length)

    # Combine them into one dataset with labels
    data = correct_data + incorrect_data
    labels = [1]*len(correct_data) + [0]*len(incorrect_data)

    # Shuffle data and labels in unison
    combined = list(zip(data, labels))
    random.shuffle(combined)
    data[:], labels[:] = zip(*combined)

    return data, labels

def sequences_to_one_hot(sequences, alphabet):
    alphabet_index = {char: i for i, char in enumerate(alphabet)}
    max_length = max(len(seq) for seq in sequences) if sequences else 1  # Ensure at least one time step

    one_hot_tensor = torch.zeros((len(sequences), max_length, len(alphabet)), dtype=torch.float32)
    for i, seq in enumerate(sequences):
        for j, char in enumerate(seq):
            one_hot_tensor[i, j, alphabet_index[char]] = 1

    sequence_lengths = torch.tensor([len(seq) if len(seq) > 0 else 1 for seq in sequences], dtype=torch.int64)
    return one_hot_tensor, sequence_lengths


class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, nonlinearity='tanh', batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, lengths):
        x_packed = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        packed_output, _ = self.rnn(x_packed)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        output = self.fc(output)
        output = torch.sigmoid(output[:, -1, :])
        return output

def train(model, criterion, optimizer, data, lengths, labels, epochs=10):
    model.train()
    loss_vector = []
    gradient_norms = []

    for epoch in range(epochs):
        epoch_gradients = []
        for i in range(len(data)):
            inputs = data[i].unsqueeze(0).to(device)
            seq_length = lengths[i].unsqueeze(0)  # Keep this tensor on CPU
            target = torch.tensor([labels[i]], dtype=torch.float).unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(inputs, seq_length)
            loss = criterion(outputs, target)
            loss.backward()
            gradients = torch.sqrt(sum(p.grad.norm()**2 for p in model.parameters() if p.grad is not None))
            epoch_gradients.append(gradients.item())
            optimizer.step()

        loss_vector.append(loss.item())
        gradient_norms.append(epoch_gradients)

    return loss_vector, gradient_norms

def test(model, data, lengths, labels):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for i in range(len(data)):
            inputs = data[i].unsqueeze(0).to(device)
            seq_length = lengths[i].unsqueeze(0)  # Keep this tensor on CPU
            outputs = model(inputs, seq_length)
            predicted = outputs.round()
            total += 1
            correct += (predicted.item() == labels[i])

    accuracy = correct / total
    return accuracy


# Parameters
REGEX = '(0|1)*0{2,4}(0|1)*1{2,4}'
ALPHABET = ['0','1']
PROPORTION = 0.5
NUM_SAMPLES = 1000
TRAIN_RATIO = 0.8
INPUT_SIZE = len(ALPHABET)  # Adjusted for one-hot encoding
HIDDEN_SIZE = 3
OUTPUT_SIZE = 1

# Sequence lengths

#long
#seq_lengths = [2,3,4,5,6,7,8,9,10,100,200,300,400,500,600,700,800,900, 1000,2000,3000,4000,5000,6000,7000,8000,9000,10000]
#HIDDEN_SIZES = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]

#short
seq_lengths = [2, 5, 8, 11, 14, 17, 20, 23, 26]
HIDDEN_SIZES = [2, 5, 8, 11, 14, 17, 20, 23, 26]


# Store results
results = []

/tmp/ipykernel_1251/156770665.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Using device: cuda


In [2]:
for seq_length in seq_lengths:
    for hidden_size in HIDDEN_SIZES:
        print("\nTraining Segment for Sequence Length", seq_length, "and Hidden Size", hidden_size)
        print("=======================================")

        data, labels = generate_data(REGEX, ALPHABET, NUM_SAMPLES, seq_length, PROPORTION)
        data, sequence_lengths = sequences_to_one_hot(data, ALPHABET)

        # Split data into training and testing
        split_index = int(len(data) * TRAIN_RATIO)
        train_data, test_data = data[:split_index], data[split_index:]
        train_lengths, test_lengths = sequence_lengths[:split_index], sequence_lengths[split_index:]
        train_labels, test_labels = labels[:split_index], labels[split_index:]

        model = SimpleRNN(INPUT_SIZE, hidden_size, OUTPUT_SIZE).to(device)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        loss_vector, gradient_norms = train(model, criterion, optimizer, train_data, train_lengths, train_labels)
        accuracy = test(model, test_data, test_lengths, test_labels)

        avg_gradient_norms = [np.mean(epoch) for epoch in zip(*gradient_norms)]
        results.append({'Sequence Length': seq_length,
                        'Hidden Size': hidden_size,
                        'Loss Vector': loss_vector, 
                        'Accuracy': accuracy
                        #,'Average Gradient Norms': avg_gradient_norms
        })
        print(f'Sequence Length: {seq_length}, Hidden Size: {hidden_size}, Accuracy: {accuracy}')

# Save results to CSV
df = pd.DataFrame(results)
df.to_csv('shortsequence_nonoise_(0|1)*0{2,4}(0|1)*1{2,4}.csv', index=False)


Training Segment for Sequence Length 2 and Hidden Size 2
Sequence Length: 2, Hidden Size: 2, Accuracy: 0.835

Training Segment for Sequence Length 2 and Hidden Size 5
Sequence Length: 2, Hidden Size: 5, Accuracy: 0.805

Training Segment for Sequence Length 2 and Hidden Size 8
Sequence Length: 2, Hidden Size: 8, Accuracy: 0.805

Training Segment for Sequence Length 2 and Hidden Size 11
Sequence Length: 2, Hidden Size: 11, Accuracy: 0.85

Training Segment for Sequence Length 2 and Hidden Size 14
Sequence Length: 2, Hidden Size: 14, Accuracy: 0.825

Training Segment for Sequence Length 2 and Hidden Size 17
Sequence Length: 2, Hidden Size: 17, Accuracy: 0.84

Training Segment for Sequence Length 2 and Hidden Size 20
Sequence Length: 2, Hidden Size: 20, Accuracy: 0.835

Training Segment for Sequence Length 2 and Hidden Size 23
Sequence Length: 2, Hidden Size: 23, Accuracy: 0.825

Training Segment for Sequence Length 2 and Hidden Size 26
Sequence Length: 2, Hidden Size: 26, Accuracy: 0.795


In [ ]:
data, labels = generate_data(REGEX, ALPHABET, 50, 10, PROPORTION)

In [ ]:
for i in range(len(data)):
    print(data[i], labels[i])

In [5]:
# Example usage
data = """
Training Segment for Sequence Length 2 and Hidden Size 2
=======================================
Sequence Length: 2, Hidden Size: 2, Accuracy: 0.795

Training Segment for Sequence Length 2 and Hidden Size 5
=======================================
Sequence Length: 2, Hidden Size: 5, Accuracy: 0.83

Training Segment for Sequence Length 2 and Hidden Size 8
=======================================
Sequence Length: 2, Hidden Size: 8, Accuracy: 0.815

Training Segment for Sequence Length 2 and Hidden Size 11
=======================================
Sequence Length: 2, Hidden Size: 11, Accuracy: 0.785

Training Segment for Sequence Length 2 and Hidden Size 14
=======================================
Sequence Length: 2, Hidden Size: 14, Accuracy: 0.865

Training Segment for Sequence Length 2 and Hidden Size 17
=======================================
Sequence Length: 2, Hidden Size: 17, Accuracy: 0.865

Training Segment for Sequence Length 2 and Hidden Size 20
=======================================
Sequence Length: 2, Hidden Size: 20, Accuracy: 0.845

Training Segment for Sequence Length 2 and Hidden Size 23
=======================================
Sequence Length: 2, Hidden Size: 23, Accuracy: 0.835

Training Segment for Sequence Length 2 and Hidden Size 26
=======================================
Sequence Length: 2, Hidden Size: 26, Accuracy: 0.755

Training Segment for Sequence Length 5 and Hidden Size 2
=======================================
Sequence Length: 5, Hidden Size: 2, Accuracy: 0.545

Training Segment for Sequence Length 5 and Hidden Size 5
=======================================
Sequence Length: 5, Hidden Size: 5, Accuracy: 0.91

Training Segment for Sequence Length 5 and Hidden Size 8
=======================================
Sequence Length: 5, Hidden Size: 8, Accuracy: 0.91

Training Segment for Sequence Length 5 and Hidden Size 11
=======================================
Sequence Length: 5, Hidden Size: 11, Accuracy: 0.9

Training Segment for Sequence Length 5 and Hidden Size 14
=======================================
Sequence Length: 5, Hidden Size: 14, Accuracy: 0.925

Training Segment for Sequence Length 5 and Hidden Size 17
=======================================
Sequence Length: 5, Hidden Size: 17, Accuracy: 0.91

Training Segment for Sequence Length 5 and Hidden Size 20
=======================================
Sequence Length: 5, Hidden Size: 20, Accuracy: 0.925

Training Segment for Sequence Length 5 and Hidden Size 23
=======================================
Sequence Length: 5, Hidden Size: 23, Accuracy: 0.895

Training Segment for Sequence Length 5 and Hidden Size 26
=======================================
Sequence Length: 5, Hidden Size: 26, Accuracy: 0.94

Training Segment for Sequence Length 8 and Hidden Size 2
=======================================
Sequence Length: 8, Hidden Size: 2, Accuracy: 0.46

Training Segment for Sequence Length 8 and Hidden Size 5
=======================================
Sequence Length: 8, Hidden Size: 5, Accuracy: 0.915

Training Segment for Sequence Length 8 and Hidden Size 8
=======================================
Sequence Length: 8, Hidden Size: 8, Accuracy: 0.925

Training Segment for Sequence Length 8 and Hidden Size 11
=======================================
Sequence Length: 8, Hidden Size: 11, Accuracy: 0.92

Training Segment for Sequence Length 8 and Hidden Size 14
=======================================
Sequence Length: 8, Hidden Size: 14, Accuracy: 0.94

Training Segment for Sequence Length 8 and Hidden Size 17
=======================================
Sequence Length: 8, Hidden Size: 17, Accuracy: 0.92

Training Segment for Sequence Length 8 and Hidden Size 20
=======================================
Sequence Length: 8, Hidden Size: 20, Accuracy: 0.95

Training Segment for Sequence Length 8 and Hidden Size 23
=======================================
Sequence Length: 8, Hidden Size: 23, Accuracy: 0.485

Training Segment for Sequence Length 8 and Hidden Size 26
=======================================
Sequence Length: 8, Hidden Size: 26, Accuracy: 0.92

Training Segment for Sequence Length 11 and Hidden Size 2
=======================================
Sequence Length: 11, Hidden Size: 2, Accuracy: 0.475

Training Segment for Sequence Length 11 and Hidden Size 5
=======================================
Sequence Length: 11, Hidden Size: 5, Accuracy: 0.5

Training Segment for Sequence Length 11 and Hidden Size 8
=======================================
Sequence Length: 11, Hidden Size: 8, Accuracy: 0.875

Training Segment for Sequence Length 11 and Hidden Size 11
=======================================
Sequence Length: 11, Hidden Size: 11, Accuracy: 0.91

Training Segment for Sequence Length 11 and Hidden Size 14
=======================================
Sequence Length: 11, Hidden Size: 14, Accuracy: 0.89

Training Segment for Sequence Length 11 and Hidden Size 17
=======================================
Sequence Length: 11, Hidden Size: 17, Accuracy: 0.935

Training Segment for Sequence Length 11 and Hidden Size 20
=======================================
Sequence Length: 11, Hidden Size: 20, Accuracy: 0.955

Training Segment for Sequence Length 11 and Hidden Size 23
=======================================
Sequence Length: 11, Hidden Size: 23, Accuracy: 0.945

Training Segment for Sequence Length 11 and Hidden Size 26
=======================================
Sequence Length: 11, Hidden Size: 26, Accuracy: 0.895

Training Segment for Sequence Length 14 and Hidden Size 2
=======================================
Sequence Length: 14, Hidden Size: 2, Accuracy: 0.525

Training Segment for Sequence Length 14 and Hidden Size 5
=======================================
Sequence Length: 14, Hidden Size: 5, Accuracy: 0.89

Training Segment for Sequence Length 14 and Hidden Size 8
=======================================
Sequence Length: 14, Hidden Size: 8, Accuracy: 0.87

Training Segment for Sequence Length 14 and Hidden Size 11
=======================================
Sequence Length: 14, Hidden Size: 11, Accuracy: 0.86

Training Segment for Sequence Length 14 and Hidden Size 14
=======================================
Sequence Length: 14, Hidden Size: 14, Accuracy: 0.84

Training Segment for Sequence Length 14 and Hidden Size 17
=======================================
Sequence Length: 14, Hidden Size: 17, Accuracy: 0.89

Training Segment for Sequence Length 14 and Hidden Size 20
=======================================
Sequence Length: 14, Hidden Size: 20, Accuracy: 0.46

Training Segment for Sequence Length 14 and Hidden Size 23
=======================================
Sequence Length: 14, Hidden Size: 23, Accuracy: 0.505

Training Segment for Sequence Length 14 and Hidden Size 26
=======================================
Sequence Length: 14, Hidden Size: 26, Accuracy: 0.555

Training Segment for Sequence Length 17 and Hidden Size 2
=======================================
Sequence Length: 17, Hidden Size: 2, Accuracy: 0.495

Training Segment for Sequence Length 17 and Hidden Size 5
=======================================
Sequence Length: 17, Hidden Size: 5, Accuracy: 0.81

Training Segment for Sequence Length 17 and Hidden Size 8
=======================================
Sequence Length: 17, Hidden Size: 8, Accuracy: 0.48

Training Segment for Sequence Length 17 and Hidden Size 11
=======================================
Sequence Length: 17, Hidden Size: 11, Accuracy: 0.54

Training Segment for Sequence Length 17 and Hidden Size 14
=======================================
Sequence Length: 17, Hidden Size: 14, Accuracy: 0.435

Training Segment for Sequence Length 17 and Hidden Size 17
=======================================
Sequence Length: 17, Hidden Size: 17, Accuracy: 0.84

Training Segment for Sequence Length 17 and Hidden Size 20
=======================================
Sequence Length: 17, Hidden Size: 20, Accuracy: 0.485

Training Segment for Sequence Length 17 and Hidden Size 23
=======================================
Sequence Length: 17, Hidden Size: 23, Accuracy: 0.545

Training Segment for Sequence Length 17 and Hidden Size 26
=======================================
Sequence Length: 17, Hidden Size: 26, Accuracy: 0.905

Training Segment for Sequence Length 20 and Hidden Size 2
=======================================
Sequence Length: 20, Hidden Size: 2, Accuracy: 0.535

Training Segment for Sequence Length 20 and Hidden Size 5
=======================================
Sequence Length: 20, Hidden Size: 5, Accuracy: 0.62

Training Segment for Sequence Length 20 and Hidden Size 8
=======================================
Sequence Length: 20, Hidden Size: 8, Accuracy: 0.6

Training Segment for Sequence Length 20 and Hidden Size 11
=======================================
Sequence Length: 20, Hidden Size: 11, Accuracy: 0.47

Training Segment for Sequence Length 20 and Hidden Size 14
=======================================
Sequence Length: 20, Hidden Size: 14, Accuracy: 0.61

Training Segment for Sequence Length 20 and Hidden Size 17
=======================================
Sequence Length: 20, Hidden Size: 17, Accuracy: 0.515

Training Segment for Sequence Length 20 and Hidden Size 20
=======================================
Sequence Length: 20, Hidden Size: 20, Accuracy: 0.54

Training Segment for Sequence Length 20 and Hidden Size 23
=======================================
Sequence Length: 20, Hidden Size: 23, Accuracy: 0.545

Training Segment for Sequence Length 20 and Hidden Size 26
=======================================
Sequence Length: 20, Hidden Size: 26, Accuracy: 0.7

Training Segment for Sequence Length 23 and Hidden Size 2
=======================================
Sequence Length: 23, Hidden Size: 2, Accuracy: 0.655

Training Segment for Sequence Length 23 and Hidden Size 5
=======================================
Sequence Length: 23, Hidden Size: 5, Accuracy: 0.65

Training Segment for Sequence Length 23 and Hidden Size 8
=======================================
Sequence Length: 23, Hidden Size: 8, Accuracy: 0.625

Training Segment for Sequence Length 23 and Hidden Size 11
=======================================
Sequence Length: 23, Hidden Size: 11, Accuracy: 0.755

Training Segment for Sequence Length 23 and Hidden Size 14
=======================================
Sequence Length: 23, Hidden Size: 14, Accuracy: 0.715

Training Segment for Sequence Length 23 and Hidden Size 17
=======================================
Sequence Length: 23, Hidden Size: 17, Accuracy: 0.715

Training Segment for Sequence Length 23 and Hidden Size 20
=======================================
Sequence Length: 23, Hidden Size: 20, Accuracy: 0.485

Training Segment for Sequence Length 23 and Hidden Size 23
=======================================
Sequence Length: 23, Hidden Size: 23, Accuracy: 0.765

Training Segment for Sequence Length 23 and Hidden Size 26
=======================================
Sequence Length: 23, Hidden Size: 26, Accuracy: 0.705

Training Segment for Sequence Length 26 and Hidden Size 2
=======================================
Sequence Length: 26, Hidden Size: 2, Accuracy: 0.785

Training Segment for Sequence Length 26 and Hidden Size 5
=======================================
Sequence Length: 26, Hidden Size: 5, Accuracy: 0.765

Training Segment for Sequence Length 26 and Hidden Size 8
=======================================
Sequence Length: 26, Hidden Size: 8, Accuracy: 0.755

Training Segment for Sequence Length 26 and Hidden Size 11
=======================================
Sequence Length: 26, Hidden Size: 11, Accuracy: 0.75

Training Segment for Sequence Length 26 and Hidden Size 14
=======================================
Sequence Length: 26, Hidden Size: 14, Accuracy: 0.52

Training Segment for Sequence Length 26 and Hidden Size 17
=======================================
Sequence Length: 26, Hidden Size: 17, Accuracy: 0.79

Training Segment for Sequence Length 26 and Hidden Size 20
=======================================
Sequence Length: 26, Hidden Size: 20, Accuracy: 0.755

Training Segment for Sequence Length 26 and Hidden Size 23
=======================================
Sequence Length: 26, Hidden Size: 23, Accuracy: 0.75

Training Segment for Sequence Length 26 and Hidden Size 26
=======================================
"""

In [4]:
import re

def parse_training_data(data):
    # Define the regular expression pattern for extracting data
    pattern = r"Sequence Length: (\d+), Hidden Size: (\d+), Accuracy: (\d\.\d+)"

    # Find all matches in the data string
    matches = re.findall(pattern, data)

    # Convert matches to a list of dictionaries
    parsed_data = []
    for match in matches:
        sequence_length, hidden_size, accuracy = match
        parsed_data.append({
            'Sequence Length': int(sequence_length),
            'Hidden Size': int(hidden_size),
            'Accuracy': float(accuracy)
        })

    return parsed_data

parsed_data = parse_training_data(data)
for segment in parsed_data:
    print(segment)

{'Sequence Length': 2, 'Hidden Size': 2, 'Accuracy': 0.795}
{'Sequence Length': 2, 'Hidden Size': 5, 'Accuracy': 0.83}
{'Sequence Length': 2, 'Hidden Size': 8, 'Accuracy': 0.815}
{'Sequence Length': 2, 'Hidden Size': 11, 'Accuracy': 0.785}
{'Sequence Length': 2, 'Hidden Size': 14, 'Accuracy': 0.865}
{'Sequence Length': 2, 'Hidden Size': 17, 'Accuracy': 0.865}
{'Sequence Length': 2, 'Hidden Size': 20, 'Accuracy': 0.845}
{'Sequence Length': 2, 'Hidden Size': 23, 'Accuracy': 0.835}
{'Sequence Length': 2, 'Hidden Size': 26, 'Accuracy': 0.755}
{'Sequence Length': 5, 'Hidden Size': 2, 'Accuracy': 0.545}
{'Sequence Length': 5, 'Hidden Size': 5, 'Accuracy': 0.91}
{'Sequence Length': 5, 'Hidden Size': 8, 'Accuracy': 0.91}
{'Sequence Length': 5, 'Hidden Size': 11, 'Accuracy': 0.9}
{'Sequence Length': 5, 'Hidden Size': 14, 'Accuracy': 0.925}
{'Sequence Length': 5, 'Hidden Size': 17, 'Accuracy': 0.91}
{'Sequence Length': 5, 'Hidden Size': 20, 'Accuracy': 0.925}
{'Sequence Length': 5, 'Hidden Size'